# Modelling More Realistic Corrosion

In [4]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import sympy
import scipy

$$\require{mhchem}$$

## Mass transfer limited corrosion

In the first attempt at modelling corrosion, I used a thickness loss rate per year for steel in atmospheric corrosion to calculate backwards to moles per second of iron oxidation. This is not a very realistic or physical basis for many reasons, not least of which is that atmospheric corrosion is carried out over long periods of time, where the metal is exposed to dry, humid, rain, hot, cold, etc. conditions as the weather and seasons vary. 

I would this time instead like to try to start from a more real system. I am thinking in this case carbon steel under a layer of salt water where there corrosion rate is limited by mass transfer of oxygen. Carbon steel corrodes readily in oxygen rich salt water, and as oxygen is reduced in the corrosion reaction an oxygen concentration gradient forms between the bulk solution and the metal surface. Assuming the oxygen is consumed faster than is can be transported to the surface, then the oxygen concentration at the surface will eventually reach zero, since it will be consumed instantly as it reaches it. The space this concentration gradient extends from the metal surface is called the Nernst layer. 

### Calculations for corrosion rate

[The Engineering ToolBox](https://www.engineeringtoolbox.com/oxygen-solubility-water-d_841.html) provides oxygen solubility at different temperatures in sea water, which has a salinity of 35 ppt, or 35 g salt per kg sea water. At 25&deg; C and 1 bar, oxygen has a solubility of about 206 &mu;M. Since the partial pressure of oxygen in air is 20%, the concentration of oxygen in the bulk solution would be about 41.2 &mu;M, or 4.12E-5 M. 

Transport of oxygen through the Nernst layer to the surface of the metal can be determined with Fick's law.

$$
J_{\ce{O2}} = -D_{\ce{O2}}\frac{dc_{\ce{O2}}}{dx}
$$

Here $J_{\ce{O2}}$ is the flux of $\ce{O2}$ to the metal surface in $\frac{mol}{cm^2s}$, $D_{\ce{O2}}$ is the diffusion coefficient of $\ce{O2}$ in $\frac{cm^2}{s}$, $c_{\ce{O2}}$ is the concentration of $\ce{O2}$ in $\frac{mol}{cm^3}$, and $x$ is the distance from the metal surface in cm. The equation can then be integrated from $x = 0$ to $x = \delta$ which gives

$$
J_{\ce{O2}} = -D_{\ce{O2}}\frac{c_{\ce{O2},b} - c_{\ce{O2},0}}{\delta}
$$

where $c_{\ce{O2},b}$ and $c_{\ce{O2},0}$ are the bulk and metal surface concentrations of $\ce{O2}$, respectively, and $\delta$ is the thickness of the Nernst layer in cm.